# Analyzing 311 Street Flooding Complaints in NYC for Tropical Storm Elsa (7/9) as well as the Flash Flood the day before (7/8) 
### Note: complaint duration is capped on 7/12

Mark Bauer  

In [143]:
# importing libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
import os

In [144]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [145]:
%watermark -v -p numpy,pandas

CPython 3.7.1
IPython 7.20.0

numpy 1.19.2
pandas 1.2.1


In [146]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

# socrata token
# register for one here: https://dev.socrata.com/docs/app-tokens.html
socrata_token = os.environ.get("SODAPY_APPTOKEN")

In [147]:
client = Socrata(socrata_domain, socrata_token)

# good practice to review the columns
metadata = client.get_metadata(socrata_dataset_identifier)
[x['fieldName'] for x in metadata['columns']]

['unique_key',
 'created_date',
 'closed_date',
 'agency',
 'agency_name',
 'complaint_type',
 'descriptor',
 'location_type',
 'incident_zip',
 'incident_address',
 'street_name',
 'cross_street_1',
 'cross_street_2',
 'intersection_street_1',
 'intersection_street_2',
 'address_type',
 'city',
 'landmark',
 'facility_type',
 'status',
 'due_date',
 'resolution_description',
 'resolution_action_updated_date',
 'community_board',
 'bbl',
 'borough',
 'x_coordinate_state_plane',
 'y_coordinate_state_plane',
 'open_data_channel_type',
 'park_facility_name',
 'park_borough',
 'vehicle_type',
 'taxi_company_borough',
 'taxi_pick_up_location',
 'bridge_highway_name',
 'bridge_highway_direction',
 'road_ramp',
 'bridge_highway_segment',
 'latitude',
 'longitude',
 'location',
 ':@computed_region_efsh_h5xi',
 ':@computed_region_f5dn_yrer',
 ':@computed_region_yeji_bk3q',
 ':@computed_region_92fq_4b7q',
 ':@computed_region_sbqj_enih']

# exploring the full dataset

## checking for the word 'flood' in the descriptor column

In [148]:
client = Socrata("data.cityofnewyork.us", socrata_token, timeout=1000)

query = """
SELECT 
    descriptor, count(descriptor) AS count
WHERE 
    LOWER(descriptor) LIKE '%flood%'
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
"""

results = client.get("erm2-nwe9", query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print(results_df.shape)
results_df

(11, 2)


,descriptor,count
0,Catch Basin Clogged/Flooding (Use Comments) (SC),92577
1,Street Flooding (SJ),28748
2,Flood Light Lamp Out,6042
3,Highway Flooding (SH),2871
4,Flood Light Lamp Cycling,2522
5,Ready NY - Flooding,271
6,Flood Light Lamp Dayburning,207
7,Flood Light Lamp Missing,193
8,Flood Light Lamp Dim,178
9,RAIN GARDEN FLOODING (SRGFLD),107


## checking for all the descriptor values where complaint_type = 'Sewer'

In [149]:
client = Socrata("data.cityofnewyork.us", socrata_token, timeout=1000)

query = """
SELECT 
    descriptor, count(descriptor) AS count
WHERE 
    complaint_type='Sewer'
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
"""

results = client.get("erm2-nwe9", query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print(results_df.shape)
results_df.head(len(results_df))

(27, 2)


,descriptor,count
0,Sewer Backup (Use Comments) (SA),154364
1,Catch Basin Clogged/Flooding (Use Comments) (SC),92577
2,Catch Basin Sunken/Damaged/Raised (SC1),29260
3,Street Flooding (SJ),28748
4,Manhole Cover Broken/Making Noise (SB),19793
5,Manhole Cover Missing (Emergency) (SA3),18083
6,Sewer Odor (SA2),15935
7,Defective/Missing Curb Piece (SC4),8642
8,Manhole Overflow (Use Comments) (SA1),7181
9,Catch Basin Search (SC2),4154


# query data for only street flooding complaints

In [150]:
client = Socrata("data.cityofnewyork.us", socrata_token, timeout=1000)

# query all columns as well as extracting date information as new columns

query = """
SELECT 
    *,
    date_extract_d(created_date) AS day,
    date_extract_dow(created_date) AS day_of_week,
    date_extract_hh(created_date) AS hour,
    date_extract_mm(created_date) AS minute,
    date_trunc_ymd(created_date) AS year_month_day
WHERE 
    descriptor = 'Street Flooding (SJ)'
LIMIT 
    1000000
"""

results = client.get("erm2-nwe9", query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# format new date column
results_df['year_month_day'] = results_df['year_month_day'].str.split('T').str[0]

print(results_df.shape)
results_df.head()

(28748, 37)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,...,location,day,day_of_week,hour,minute,year_month_day,intersection_street_1,intersection_street_2,facility_type,due_date
0,50465779,2021-05-04T17:58:00.000,2021-05-05T07:15:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11226,2225 TILDEN AVENUE,TILDEN AVENUE,...,"{'latitude': '40.64660766955987', 'longitude':...",4,2,17,58,2021-05-04,NaN,NaN,NaN,NaN
1,50471923,2021-05-04T10:25:00.000,2021-05-05T08:45:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11234,1477 EAST 51 STREET,EAST 51 STREET,...,"{'latitude': '40.62449690722713', 'longitude':...",4,2,10,25,2021-05-04,NaN,NaN,NaN,NaN
2,45123962,2019-12-10T12:13:00.000,2019-12-11T11:45:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11434,NaN,NaN,...,"{'latitude': '40.66016771824149', 'longitude':...",10,2,12,13,2019-12-10,147 AVENUE,ROCKAWAY BOULEVARD,NaN,NaN
3,45123981,2019-12-10T16:23:00.000,2019-12-11T08:30:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10022,NaN,NaN,...,"{'latitude': '40.761668277085114', 'longitude'...",10,2,16,23,2019-12-10,5 AVENUE,WEST 55 STREET,NaN,NaN
4,45123982,2019-12-10T14:30:00.000,2019-12-15T09:20:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11205,NaN,NaN,...,"{'latitude': '40.69909044367837', 'longitude':...",10,2,14,30,2019-12-10,FLUSHING AVENUE,SPENCER STREET,NaN,NaN


In [151]:
print('Number of total records: {:,}.'.format(len(results_df)))

print('\ndate information:')
print('Min: {}.'.format(results_df.created_date.min()))
print('Max: {}'.format(results_df.created_date.max()))

Number of total records: 28,748.

date information:
Min: 2010-01-02T08:26:00.000.
Max: 2021-07-14T23:05:00.000


In [152]:
# saving only data for hurricane elsa
elsa_df = results_df.loc[(results_df["year_month_day"] > '2021-07-07') & 
                         (results_df["year_month_day"] < '2021-07-13')]

# reset index
elsa_df = elsa_df.reset_index(drop=True)

print(elsa_df.shape)
elsa_df.head()

(190, 37)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,...,location,day,day_of_week,hour,minute,year_month_day,intersection_street_1,intersection_street_2,facility_type,due_date
0,51143675,2021-07-08T16:04:00.000,2021-07-09T10:30:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10471,6485 BROADWAY,BROADWAY,...,"{'latitude': '40.905351378307486', 'longitude'...",8,4,16,4,2021-07-08,NaN,NaN,NaN,NaN
1,51145094,2021-07-08T16:00:00.000,2021-07-08T20:45:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10034,200 NAGLE AVENUE,NAGLE AVENUE,...,"{'latitude': '40.86210236351487', 'longitude':...",8,4,16,0,2021-07-08,NaN,NaN,NaN,NaN
2,51146482,2021-07-08T21:04:00.000,NaN,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11236,1035 EAST 102 STREET,EAST 102 STREET,...,"{'latitude': '40.6437429889248', 'longitude': ...",8,4,21,4,2021-07-08,NaN,NaN,NaN,NaN
3,51148924,2021-07-08T17:04:00.000,2021-07-09T07:44:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10471,NaN,NaN,...,"{'latitude': '40.900731405929164', 'longitude'...",8,4,17,4,2021-07-08,FIELDSTON ROAD,WEST 253 STREET,NaN,NaN
4,51139519,2021-07-08T16:44:00.000,2021-07-09T17:15:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11432,172-39 HIGHLAND AVENUE,HIGHLAND AVENUE,...,"{'latitude': '40.71338015366497', 'longitude':...",8,4,16,44,2021-07-08,NaN,NaN,NaN,NaN


In [153]:
print('Number of total records: {:,}.'.format(len(elsa_df))) 

print('\ndate information:')
print('Min: {}.'.format(elsa_df.created_date.min()))
print('Max: {}'.format(elsa_df.created_date.max()))

Number of total records: 190.

date information:
Min: 2021-07-08T08:17:00.000.
Max: 2021-07-12T20:31:00.000


In [154]:
print('number of complaints not closed: {}.'.format(elsa_df['closed_date'].isnull().sum()))
print('% of total: {:.2f}%.'.format(elsa_df['closed_date'].isnull().sum() / len(elsa_df) * 100))

number of complaints not closed: 53.
% of total: 27.89%.


In [155]:
# concise summary of a DataFrame
elsa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   unique_key                      190 non-null    object
 1   created_date                    190 non-null    object
 2   closed_date                     137 non-null    object
 3   agency                          190 non-null    object
 4   agency_name                     190 non-null    object
 5   complaint_type                  190 non-null    object
 6   descriptor                      190 non-null    object
 7   incident_zip                    190 non-null    object
 8   incident_address                151 non-null    object
 9   street_name                     151 non-null    object
 10  cross_street_1                  149 non-null    object
 11  cross_street_2                  149 non-null    object
 12  address_type                    190 non-null    ob

In [156]:
# concise summary of a DataFrame with sample value
T = elsa_df.head(1).T
T.rename(columns={0:'sample_value'})

,sample_value
unique_key,51143675
created_date,2021-07-08T16:04:00.000
closed_date,2021-07-09T10:30:00.000
agency,DEP
agency_name,Department of Environmental Protection
complaint_type,Sewer
descriptor,Street Flooding (SJ)
incident_zip,10471
incident_address,6485 BROADWAY
street_name,BROADWAY


In [157]:
# exporting data as csv
elsa_df.to_csv('elsa_311_calls.csv', index=False)

In [158]:
# sanity check
%ls

CODE_OF_CONDUCT.md    README.md             elsa_311_calls.csv
LICENSE               download_data.png     query_tutorial.ipynb


In [159]:
# sanity check
test = pd.read_csv('elsa_311_calls.csv')

print('Number of total records: {:,}.\n'.format(len(test)))      
print('Min: {}.'.format(test.created_date.min()))
print('Max: {}'.format(test.created_date.max()))

test.head()

Number of total records: 190.

Min: 2021-07-08T08:17:00.000.
Max: 2021-07-12T20:31:00.000


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,...,location,day,day_of_week,hour,minute,year_month_day,intersection_street_1,intersection_street_2,facility_type,due_date
0,51143675,2021-07-08T16:04:00.000,2021-07-09T10:30:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10471,6485 BROADWAY,BROADWAY,...,"{'latitude': '40.905351378307486', 'longitude'...",8,4,16,4,2021-07-08,NaN,NaN,NaN,NaN
1,51145094,2021-07-08T16:00:00.000,2021-07-08T20:45:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10034,200 NAGLE AVENUE,NAGLE AVENUE,...,"{'latitude': '40.86210236351487', 'longitude':...",8,4,16,0,2021-07-08,NaN,NaN,NaN,NaN
2,51146482,2021-07-08T21:04:00.000,NaN,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11236,1035 EAST 102 STREET,EAST 102 STREET,...,"{'latitude': '40.6437429889248', 'longitude': ...",8,4,21,4,2021-07-08,NaN,NaN,NaN,NaN
3,51148924,2021-07-08T17:04:00.000,2021-07-09T07:44:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10471,NaN,NaN,...,"{'latitude': '40.900731405929164', 'longitude'...",8,4,17,4,2021-07-08,FIELDSTON ROAD,WEST 253 STREET,NaN,NaN
4,51139519,2021-07-08T16:44:00.000,2021-07-09T17:15:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11432,172-39 HIGHLAND AVENUE,HIGHLAND AVENUE,...,"{'latitude': '40.71338015366497', 'longitude':...",8,4,16,44,2021-07-08,NaN,NaN,NaN,NaN
